# Twitter Scraper

In [1]:
!pip install msedge-selenium-tools selenium==3.141

You should consider upgrading via the 'c:\users\kdw27\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


In [12]:
import re
import csv
import time
from getpass import getpass
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions
import pandas as pd
from dateutil.parser import parse

def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return

    text = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text

    
    tweet = (username, handle, postdate, text)
    return tweet    

In [122]:
# Open Twitter with Webdriver
'''
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)
'''
path='chromedriver.exe'
driver = webdriver.Chrome(path)
chrome_options= webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
# Navigate to the search page
driver.get('https://twitter.com/search?f=live&q=%EC%97%AC%ED%96%89%20-filter%3Alinks%20-filter%3Areplies&src=typed_query')
'''Be sure to add your stopwords in advanced search settings'''
# driver.maximize_window()

In [123]:
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[2]/div/div/section/div/div/div[1]/div/div/article')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # End of scroll (Stop condition)
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=96.0.4664.45)


# Errors
- Out of Memory
      Remember the date when stops and start crawling from that date
- I keep getting Replies from Twitter
      Checkout the Twitter search page where you can not see the replies to your search result

In [126]:
data

[('メイビス',
  '@Mavis100479',
  '2019-09-22T23:21:13.000Z',
  '여러분은 학교가시죠? 전 오늘 서울떠납니다. 케리어인증함 ㄹㅇ이에요\n슲ㄷㅁ.....여행가는거아니에요..\n2\n1\n5'),
 ('카스가 사키코',
  '@sakiko_kr_bot',
  '2019-09-22T22:27:55.000Z',
  '그 여행사이트에서는 우리들 히나타비 상점가는\n“고양이 씨 상점가”라고 소개되어 있는 것 같습니다.\n샤느와르도 프랑스어로 검은 고양이 씨라는 의미이기에\n어묵 파르페는 검은 고양이 파르페라고 소개 되어 있었습니다. #bot'),
 ('이웃집감점순',
  '@lesbianibnida',
  '2019-09-22T20:23:26.000Z',
  '이십대 끝자락이 되어서야 처음으로 친구들과 여행을 왔다. 애인이 밤새 만들어준 날씨 부적이 태풍을 막아주진 못했지만 마음엔 따스한 보호막을 씌워준 것 같다. 계획과 전혀 다른 길로 왔지만 우리는 내내 즐거웠고 막힘 없이 떠들었다. 이제 또 언제 친구들과 여행을 갈 수 있으려나.\n2\n17'),
 ('김눈새',
  '@itsbirdnoon',
  '2019-09-22T18:32:19.000Z',
  '@아 ㅋㅌㅋㅌ 악몽즈 단체여행... 원래 수련회건 수학여행이건 암튼 친한사람들끼리 놀러갈때는 밤에 담력테스트까지만 생각하고 뇌 굳음...........\n\n우리 담력테스트 할래? 하는순간 분위디 싸해짐\n1\n3'),
 ('동동',
  '@dongpnob',
  '2019-09-22T17:13:29.000Z',
  '인터넷으로 만난사람들이랑 해외여행 갔다온거 개웃기네 하고 웃다가 생각해보니까 나 오타쿠라서 지금까지 가족여행 빼고 다 인터넷으로 만난사람들이랑만 해외여행 다녀옴\n3'),
 ('레이안',
  '@love_gutory_',
  '2019-09-22T16:22:51.000Z',
  '지난번 여행으로 너무즐거운일이 많았던것같아요.... 배틀이란것도  해보고 다른 사람

## Saving the tweet data

In [127]:
with open('여행_2019.08.30-2019.09.22.csv', 'w', newline='', encoding='utf-8-sig') as f:
    header = ['UserName', 'Handle', 'Date', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [128]:
df = pd.read_csv('여행_2019.08.30-2019.09.22.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.to_csv("여행_2019.08.30-2019.09.22.csv",encoding='utf-8-sig')